## How do we handle Program Error?

#### What are the common causes of errors at runtime?

* invalid inputs
    * invalid data/data types
    * invalid execution mode
* invalid context
    * internet not connected.
    * file not found
    * printer/device not ready
    * permission denied.


#### What to do when such errors occur?

* There are two possible course of actions broadly.
    1. Fix the situation and Retry (Ideal but not frequent)
        * ask user to re-enter their password.
        * try to withdraw lesser amount from ATM

    2. More common scenario (Make a graceful exit)
        * avoid crash
        * save the unsaved data
        * log the problem
        * inform the user about the problem
        * exit the program

#### Who (programming element) are all involved in the process?

1. **Error** Situation
    * we need to define what is the error.
    * we also need to define any additional information

2. **Source** of the problem
    * It is a point in the code (function) where the error occured.
    * Generally we don't resolve the problem exactly where it occurs.
        * if we do, that is not an error handling.

    * Generally, the error occurs in the business layer (may be on server) and error should be presented to the user in the client layer.
        * when we try to withdraw funds from an ATM
            * authentication can be done only on the server.
            * if authentication fails, we need to inform the user.
                * server can't handle (display error) 
                    * no point is displaying it on the server
                * it has to inform to the user (ATM)
                    * ATM must handle the problem by 
                        * displaying message
                        * ejecting or seizing the credit card.


    
3. **Error Handler**
    * Another function that handles the error
    * It may
        * display error message
        * take corrective action
            * eject/seize the card.
     
    * Generally separate from the source.



#### How the entire mechanism works.

1. source realises that some problem has occured.
    * Generally we use **if** to check for this problem situation
    
2. the source must inform the handler that error has occurred.

3. the handler must check if the error occured and take corrective measure.


#### Traditional Error Handling mechanism (You already know it. You have used it.)

1. source checks for the error using if statement
    * if triangle sides are 

2. source returns the error 
    * in most language error value should match return type of function.
        * in python this may not be necessary as there is no fixed return type.
    * common values to act as error
        * False (validate)
        * 0 or -1 if function returns int.
        * NaN if function returns double.
        * None if function returns object (perimeter()/area())
       

n. The target checks for the value
    * if value is error signal:
        * take corrective action
    * else:
        * take normal action


#### Why have we used step "n" and not step 3?

* the handler of error may not be the caller of the source directly.
* there may be multiple intermediataries (function) between source and handler

#### Case Study

---
```python
# server
class BankAccount:
    def authenticate(self, password):
        pass
    def withdraw(self, amount,password):
        pass
    def deposit(self, amount):
        pass

class Bank:
    def transfer(self, source, amount,password,target):
        pass
        # get bank accounts source and target
        # withdraw from source
        #deposit to target


#client
class ATM:
    def start(self):
        show_login_menu()

    def show_login_menu():        
        show_transaction_menu()

    def show_transfer_menu():
        bank.transfer(...)

```
---

* Here is the entire workflow of how user will initiate a tranfer action with wrong password
    * ATM.start()
        * ATM.show_login_menu()
            * ATM.show_transaction_menu() <--- **ERROR HANDLER**
                * ATM.show_transfer_menu()
                    * bank.transfer(...)
                        * bank.get_account(source)
                        * bank.get_account(target)
                        * source_account.withdraw(amount,password)
                            *bankaccount.authenticate(password) <--- **ERROR SOURCE**
    

* All functions between source and handler can be considered intermediataries
    * they are not the cause of failure.
    * they can't handle the failure.

* Consider the authentication failure
    1. source of error --> BankAccount.authenticate()
        * it generates an error value (false)

    2. it reaches BankAccount.withdraw()
        * it checks for the error message (although it is not the actual handler)
            * it returns the error value to its caller

    3. it reaches Bank.transfer()
        * it checks for the error message (although it is not the actual handler)
            * it returns the error value **False** to its caller

    4. it reaches ATM.show_transfer_menu()
        * it checks for the error message (although it is not the actual handler)
            * it returns the error value **False** to its caller

    5. it reaches ATM.show_transaction_menu()
        * it checks for the error message 
            * being the real handler, it displays the error message.     

* In this example step 1 is source, and handler is step 5
* step 2-4 is redundant code where each function checks for same error and return back to the caller.
    * it is a hand-me-down approach with lots of extra and redundant code.


#### Problem with the traditional approach

1. No specific object to represent error. (0,-1,-2, None, False, NaN)
    * no tandard

2. No error details.
    * when bank.transfer() fails it returns False
        * we know it failed.
        * we don't know why it failed?
            * invalid source account?
            * invalid target account?
            * authentication failure?
            * insufficient funds?
    * triange is_valid 
        * fails if
            * triangle sides are wrong
            * object is not of type triangle
        * in both cases it returns False
            * we don't know what made it fail.

3. same statement is used for returning valid information and error
    * additional checks at each place.

4. error is hand-me-down. intermediataries are given jobs that they don't need increasing the code.



# Object Oriented Exception Handling

### 1. What is exception?

* It is a special object.
    * In python, an exception is a subclass of exception

* being an object it can contain properties indicating exception details.

* If not handled, crashes and terminates the application.

#### In python there are many different exceptions defined


In [1]:
call_a_function_that_doesnt_exist()

NameError: name 'call_a_function_that_doesnt_exist' is not defined

In [2]:
numbers=[2,3,9,2,6]
print(numbers[10])

IndexError: list index out of range

In [3]:
numbers["hi"]

TypeError: list indices must be integers or slices, not str

##### Note:

* code works fine as long as there is no problem
* once a problem occurs, it crashes and terminates the application
* in below case there is no 'JP' will be printed because it will fail for 'PK'

In [5]:
country_info=dict(IN="India", JP="Japan", FR="France")
lookups=['IN','FR','PK','JP']

for lookup in lookups:
    print(lookup, country_info[lookup])



IN India
FR France


KeyError: 'PK'

In [7]:
def sum(a,b): return a+b

print(sum(10,20))
print(sum(10,20,30)) #causes error here
print(sum(1)) #never reaches here to give this second error.

30


TypeError: sum() takes 2 positional arguments but 3 were given

#### Let us write a simple program that may have errors.

* A fortune teller application

In [18]:

def get_luck(number):
    messages=[None, "You have a great day Ahead","Fortune favours you","Be careful today","You are born to succeed"]

    if number<0:
        negativity_around_you() # we don't have this function 
    elif number==0:
        return 1/0
    else:
        return messages[number]


def lucky_number_finder(lucky_number): #intermediatry layer
    lucky_number= int(lucky_number)
    message = get_luck(lucky_number) #represents data layer   
    return message  

def fortune_teller(input_lucky_number):
    
    message = lucky_number_finder(input_lucky_number) # represents business layer.
    print(f"Prediction: {message}")
    

#### This code may give your messages without crashing for nice inputs.

* it may work as if there is no problem

In [19]:
fortune_teller(2)
fortune_teller(3)
fortune_teller(1)

Prediction: Fortune favours you
Prediction: Be careful today
Prediction: You have a great day Ahead


#### But it can also fail for so many different reasons.

In [20]:
fortune_teller(-1)

NameError: name 'negativity_around_you' is not defined

In [22]:
fortune_teller(0)

ZeroDivisionError: division by zero

In [23]:
fortune_teller(100)

IndexError: list index out of range

In [24]:
fortune_teller("Hi")

ValueError: invalid literal for int() with base 10: 'Hi'

#### 2. Handling the error

* We don't want our application to crash
    * solve it
    * shove it.


### handling the error requires two keywords ---> try-except (and optional finally)

* most other languages uses the term try-catch.
* try is a block of code that may get into trouble (get exception)
* except is a block of code where we handle the exception.
* both these blocks are connected.
    * except must follow a try
        * you can't use it indepenetly.
    * try should be followed by at least one except of finally   

* we generally write try-except block at the handler level
    * function that would handle the error
        * and not the function that would be the source 
            

---
```python
try:
    #any of the below functions may cause error
    # they can be considered error sources
    fn1()
    fn2()
    fn3()
except:
    # here we can write the error handling logic.
```
---



In [31]:
def fortune_teller(input_lucky_number):
    try:
        message = lucky_number_finder(input_lucky_number) # represents business layer.
        print(f"Prediction: {message}")
    except:
        print(f'Prediction: Your fortune looks foggy!')
    

#### The except block will be ignored if no problem occurs.

In [32]:
fortune_teller(2)
fortune_teller(1)
fortune_teller(3)

Prediction: Fortune favours you
Prediction: You have a great day Ahead
Prediction: Be careful today


#### But the code will not crash even if there is a problem.
* once we enter except, it is assumed that error is handled
    * even if we do nothing

* except has no built-in course correction
    * it simply prevents application crash

* you may like to
    * print some message
    * log the error
    * seize the credit card
    * rebuke the user for their stupidity.

In [33]:
fortune_teller(-1)
fortune_teller(11)
fortune_teller(0)
fortune_teller('Hi')

Prediction: Your fortune looks foggy!
Prediction: Your fortune looks foggy!
Prediction: Your fortune looks foggy!
Prediction: Your fortune looks foggy!


### What if we want to know the error details (message)

* we can assign an reference in except to access the exception object

In [41]:
def fortune_teller(input_lucky_number):
    try:
        message = lucky_number_finder(input_lucky_number) # represents business layer.
        print(f"Prediction: {message}")
    except Exception as ex:
        print(f'Prediction Error: {ex}')
    

In [42]:
def test_fortunes():
    fortune_teller(-1)
    fortune_teller(11)
    fortune_teller(2)
    fortune_teller(0)
    fortune_teller(1)
    fortune_teller('Hi')

In [43]:
test_fortunes()

Prediction Error: name 'negativity_around_you' is not defined
Prediction Error: list index out of range
Prediction: Fortune favours you
Prediction Error: division by zero
Prediction: You have a great day Ahead
Prediction Error: invalid literal for int() with base 10: 'Hi'


#### These message are not meaningful.

### We may want to give different treatments to different exceptions.

* each try block may be followed by multiple except blocks each handling a different exception.

In [44]:
def fortune_teller(input_lucky_number):
    try:
        message = lucky_number_finder(input_lucky_number) # represents business layer.
        print(f"Prediction: {message}")
    except NameError as ex:
        print(f'PredictionX : There is too much of negativity around you. ')
    except ZeroDivisionError as ex:
        print(f'PredictionX : You have no luck today')
    except IndexError as ex:
        print(f'PredictionX : You are over ambitious')
    except ValueError as ex:
        print(f'PredictionX : You should learn Maths')
    

In [45]:
test_fortunes()

PredictionX : There is too much of negativity around you. 
PredictionX : You are over ambitious
Prediction: Fortune favours you
PredictionX : You have no luck today
Prediction: You have a great day Ahead
PredictionX : You should learn Maths


### 3. Raising the error/exception

* so far we have cause the program failure by writing a code that fails.
    * called a function that doesn't exit
    * intentionally caused division by zero error

* that is not the right way to create the failure situation.

* To indicate the problem you should **raise** the error.

#### What is raise?

* raise is a keyword like return. 
    * it ends the current function call.
    * In most other langauges the keyword used in this case is **throw**

* return is (should be) used for valid value

* raise is used for errors.

* return returns to the immediate caller function.
    * if that function is intermediatary, we have to write second return till it reaches handler

* raise is like throwing an exception
    * it automatically propagates till it finds a matching except
    * if there no matching except, it causes program crash

* Note we haven't written any logic for error management in our intermediatary function.


### Raising an Exception.


* currently we have **misused** exsiting exceptions to represent our failures
    * most of the exceptions were not related to my problem.
* It is a good idea to define your own exceptions

```python
raise SomeError('error message if any')
```

In [46]:
def get_luck(number):
    messages=[None, "You have a great day Ahead","Fortune favours you","Be careful today","You are born to succeed"]

    if number<0:
        raise Exception("Too much of Negativity")
    elif number==0:
        #return 1/0
        raise Exception("You luck ran out. Now its 0")
    else:
        return messages[number]


def lucky_number_finder(lucky_number): #intermediatry layer
    lucky_number= int(lucky_number)
    message = get_luck(lucky_number) #represents data layer   
    return message  


def fortune_teller(input_lucky_number):
    try:
        message = lucky_number_finder(input_lucky_number) # represents business layer.
        print(f"Prediction: {message}")
    except IndexError: 
        print(f'PrefictinX: You fly too high')
    except ValueError: 
        print(f'PredictionX: You must learn Mathematics')
    except Exception as ex:
        print(f'PredictionX: {ex}')



In [47]:
test_fortunes()

PredictionX: Too much of Negativity
PrefictinX: You fly too high
Prediction: Fortune favours you
PredictionX: You luck ran out. Now its 0
Prediction: You have a great day Ahead
PredictionX: You must learn Mathematics


### 4. User defined Exceptions.

* While python provides multiple exception, we may need exception to meet my business requirement.
* We can create custom exception to handle our use cases
* our exceptions can have additional information that can help in
    * better logging
    * better management of exception

* they can also have behaviors.


#### How to create custom exceptions

* our exception should be a class inheriting from Exception base class or any of the subclass
* we can also create our exception hierarchy.

In [51]:
class FortuneError(Exception):
    def __init__(self, lucky_number, message="Fortune Error"):
        super().__init__(message)
        self.lucky_number = lucky_number


class NegativeFortuneError(FortuneError):
    pass

class AmbitionError(FortuneError):
    pass

class IgnoranceError(FortuneError):
    pass

class NoFortuneError(FortuneError):
    pass



#### Now we can use these exception in our application

* Also note, this time we are using two try-except in our code
* one at handler
* other at intermediatary.



In [54]:
def get_luck(number):
    messages=[None, "You have a great day Ahead","Fortune favours you","Be careful today","You are born to succeed"]

    if number<0:
        raise NegativeFortuneError(number,"Too much of Negativity")
    elif number==0:
        #return 1/0
        raise NoFortuneError(number,"You luck ran out. Now its 0")
    elif number>=len(messages):
        raise AmbitionError(number,"You are over ambitious")
    else:
        return messages[number]


def lucky_number_finder(lucky_number): #intermediatry layer
    try:
        lucky_number= int(lucky_number)
        message = get_luck(lucky_number) #represents data layer   
        return message
    except ValueError: 
        raise IgnoranceError(lucky_number, "You must learn Mathemetics")  


def fortune_teller(input_lucky_number):
    try:
        message = lucky_number_finder(input_lucky_number) # represents business layer.
        print(f"Prediction: {message}")
    
    except FortuneError as ex:
        print(f'Special rediction for {ex.lucky_number}: {ex}')



In [55]:
test_fortunes()

Special rediction for -1: Too much of Negativity
Special rediction for 11: You are over ambitious
Prediction: Fortune favours you
Special rediction for 0: You luck ran out. Now its 0
Prediction: You have a great day Ahead
Special rediction for Hi: You must learn Mathemetics


#### Important Notes.

1. Here we have raised Exceptions (Line 5,8,10) of different types: NegativeFortune, NoFortune etc at source
   * But we handled only the super class FortuneError 29-30.
        * all sub class exceptions can be hadled in an except block with super class.

   * It is not compulsary but a choice.
        * we can handle them separately also.

    * **IMPORTANT**
        * if you want to except a super class exception (generic) and subclass exception (specific) you must first except subclass exception (specific) and then the superclass exception (generic)
            * otherwise, the superclass except block will catch the subclass exception also
            
        * if you want to handle FortuneException and NegativeFrotuneException, you should reight the except block of NegativeFrotuneException first, followed by FrotuneException.


2. In our code we have two try blocks

    * when exception occurs on line 5 Negative fortune exception
        1. It searches for a try block in same function get_luck()
            * since there is no-try catch, it reaches the parent funciton: lucky_number_finder()

        2. lucky_nubmer_finder() has a try block but it doesn't have matching except block  for NegativeFrotuneException or it super classes (FortuneException, Exception)
            * it reaches the parent function
            * no special code needed to do this.

        3. we reach fortune_teller() 
            * it doesn't have an specific except for NegativeFrotuneException
            * but it has an except for the sueper class FortuneException.
                * exception is handled.


3. Translating the exception.

* consider line number 17 **lucky_number= int(lucky_number)**
    * it may fail with ValueError

* but we are inerested in IgnoranceError
    * we wrote a try-except to intercept the ValueError
    * we raised IgnoranceError.


## 5. Finally

* finally is a block of code that is executed no matter what.

* It is used to release external resources.

* It is also used to execute clean-up code.

* In exception scenario there is guaranty that
    * whole try block will execute
        * code may have exception
    * any except will execute
        * there may not be an error
        * there may be an error but not matching our except
    * code after try-except will execute
        * they will execute if
            * there is no error
            * there is an error but it is handled by except
        * they will not execute
            * if here is an error with except.


* what if we want to execute one code that always works. No matter what.

* finally block.


##### finally

* it is a block like except
* it must follow try
* unlike except there can be only one finally with a try
* a try must be followed by 
    * 0 or more except
    * 0 or 1 finally
    * at least one block, either try or finally

* valid cases
    * try--->except--->finally
    * try--->except
    * try--->finally
    * try--->except--->except--->except--->finally

* invalid cases
    * try (no except or finally)
    * except
    * finally
    * try--->finally-->except
        * finally must be the last block.


##### finally executes in these scenarios.
* let us consider a code


In [58]:
class ExA(Exception):pass
class ExB(Exception):pass
class ExC(Exception):pass

def action(i):

    if i==1:
        raise ExA()
    elif i==2:
        raise ExB()
    elif i==3:
        raise ExC()
    else:
        return "T"

def fn(x):
    try:
        result = action(x)# may raise ExA, ExB, ExC
        print(result, end="")
    except ExA:
        print("A", end="")
    except ExB:
        print("B", end="")
    finally:
        print("F", end="")

    print("X")

In [59]:
# case 1: no error
# flow: try->finally-->normal end

fn(5)

TFX


In [61]:
# case 2: error occurs but it is handled.
#flow: except->finally-->normal end

fn(2)

BFX


In [62]:
# case 3 : error occurs but it is NOT handled
# flow: finally ---> exit without normal end.

fn(3)


F

ExC: 